In [24]:
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
from langchain_groq import ChatGroq


In [25]:
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
import os

In [27]:
def read_pdf_files(pdf_directory):
    pdf_loader = PyPDFDirectoryLoader(pdf_directory)
    documents = pdf_loader.load()
    return documents
    

In [28]:
doc = read_pdf_files("document/")
len(doc)

340

In [29]:
def chunk_data(_docs,chunk_size=800, chunk_overlap=50):
   text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
   docs = text_splitter.split_documents(_docs)
   return docs

In [30]:
documents = chunk_data(_docs=doc)
len(documents)

1083

In [31]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [32]:
vectors = embeddings.embed_query("what is sidewalk roadmap")
len(vectors)

384

In [33]:
# from pinecone import Pinecone, ServerlessSpec

# key = os.getenv("PINECONE_API_KEY")
# pc = Pinecone(api_key=key)
# # Create a serverless index
index_name = "millionairefastlanechatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )


In [34]:
from langchain.vectorstores import Pinecone
index = Pinecone.from_documents( documents, embeddings,index_name=index_name)

In [35]:
def retrieve_query(query,k=10):
    matching_results = index.similarity_search(query, k=k)
    return matching_results

In [36]:
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq


groq_api_key = os.environ['GROQ_API_KEY']
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name="llama3-groq-70b-8192-tool-use-preview",
            temperature=0.5,
    )
chain = load_qa_chain(llm,chain_type="stuff")

In [37]:
def chatbot(query):
    matching_results = retrieve_query(query)
    response = chain.run(input_documents=matching_results,question=query)
    return response

In [39]:
query = "what is sidewalk roadmap"
answer = chatbot(query)
print(answer)

I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.
